In [5]:
import os 
import numpy as np 
import json 
import sys 
import tqdm
import trimesh 
sys.path.append("..")
from src.utils import *

In [11]:
# collect all ycb models that has .sdf files
ycb_dir = '/media/junting/SSD_data/ycb'
model_ids = os.listdir(ycb_dir)
metadata = {
    'objects': {},
}
output_dir = '../../data/ycb'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


for model_id in model_ids:
    model_name = model_id[4:]
    model_folder = os.path.join(ycb_dir, model_id)
    sdf_file = os.path.join(model_folder, f'{model_name}.sdf')
    
    if os.path.exists(sdf_file):
        metadata['objects'][model_id] = {
            'model_id': model_id,
            'model_name': model_name
        }

        # calculate the dimension of the object mesh
        # Check if there are Google meshes; else use the TSDF folder
        if "google_16k" in os.listdir(model_folder):
            mesh_type = "google_16k"
        else:
            mesh_type = "tsdf"

        if mesh_type == "google_16k":
            mesh_file = os.path.join(model_folder, "google_16k", "textured.obj")
        elif mesh_type == "tsdf":
            mesh_file = os.path.join(model_folder, "tsdf", "textured.obj")
        mesh = trimesh.load(mesh_file)
        dimension = getMeshDimensions(mesh)

        metadata['objects'][model_id]['bbox_size'] = dimension.tolist()

# save metadata
with open(os.path.join(output_dir, 'metadata.json'), 'w') as f:
    json.dump(metadata, f, indent=4)


